##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deep Exponential Family with TFP

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/deep_exponential_family_with_tfp.ipynb"><img height="32px" src="https://colab.research.google.com/img/colab_favicon.ico" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/deep_exponential_family_with_tfp.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>
<br>
<br>
<br>


Original content [this Repository](https://github.com/blei-lab/edward/blob/master/examples/deep_exponential_family.py) and [this paper](http://www.cs.toronto.edu/~lcharlin/papers/def_aistats.pdf), created by [the Blei Lab](http://www.cs.columbia.edu/~blei/)

Ported to [Tensorflow Probability](https://www.tensorflow.org/probability/) by Matthew McAteer ([`@MatthewMcAteer0`](https://twitter.com/MatthewMcAteer0)), with help from Bryan Seybold, Mike Shwe ([`@mikeshwe`](https://twitter.com/mikeshwe)), Josh Dillon, and the rest of the TFP team at  Google ([`tfprobability@tensorflow.org`](mailto:tfprobability@tensorflow.org)).

---

- Dependencies & Prerequisites
- Introduction
- Our example
  - Hyperparameters
  - Data
  - Model
  - Inference
- References

## Dependencies & Prerequisites

<div class="alert alert-success">
    Tensorflow Probability is part of the colab default runtime, <b>so you don't need to install Tensorflow or Tensorflow Probability if you're running this in the colab</b>. 
    <br>
    If you're running this notebook in Jupyter on your own machine (and you have already installed Tensorflow), you can use the following
    <br>
      <ul>
    <li> For the most recent nightly installation: <code>pip3 install -q tfp-nightly</code></li>
    <li> For the most recent stable TFP release: <code>pip3 install -q --upgrade tensorflow-probability</code></li>
    <li> For the most recent stable GPU-connected version of TFP: <code>pip3 install -q --upgrade tensorflow-probability-gpu</code></li>
    <li> For the most recent nightly GPU-connected version of TFP: <code>pip3 install -q tfp-nightly-gpu</code></li>
    </ul>
Again, if you are running this in a Colab, Tensorflow and TFP are already installed
</div>

In [0]:
#@title Imports and Global Variables  { display-mode: "form" }
!pip3 install -q observations
from __future__ import absolute_import, division, print_function

#@markdown This sets the warning status (default is `ignore`, since this notebook runs correctly)
warning_status = "ignore" #@param ["ignore", "always", "module", "once", "default", "error"]
import warnings
warnings.filterwarnings(warning_status)
with warnings.catch_warnings():
    warnings.filterwarnings(warning_status, category=DeprecationWarning)
    warnings.filterwarnings(warning_status, category=UserWarning)

from absl import flags
import csv
from datetime import datetime
import numpy as np
import os
from six.moves import urllib
import string
import time
import collections
from contextlib import contextmanager

#@markdown This sets the styles of the plotting (default is styled like plots from [FiveThirtyeight.com](https://fivethirtyeight.com/))
matplotlib_style = 'fivethirtyeight' #@param ['fivethirtyeight', 'bmh', 'ggplot', 'seaborn', 'default', 'Solarize_Light2', 'classic', 'dark_background', 'seaborn-colorblind', 'seaborn-notebook']
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
import matplotlib.axes as axes;
from matplotlib.patches import Ellipse
%matplotlib inline
import seaborn as sns; sns.set_context('notebook')
from IPython.core.pylabtools import figsize
#@markdown This sets the resolution of the plot outputs (`retina` is the highest resolution)
notebook_screen_res = 'retina' #@param ['retina', 'png', 'jpeg', 'svg', 'pdf']
%config InlineBackend.figure_format = notebook_screen_res

import tensorflow as tf
tfe = tf.contrib.eager

# Eager Execution
#@markdown Check the box below if you want to use [Eager Execution](https://www.tensorflow.org/guide/eager)
#@markdown Eager execution provides An intuitive interface, Easier debugging, and a control flow comparable to Numpy. You can read more about it on the [Google AI Blog](https://ai.googleblog.com/2017/10/eager-execution-imperative-define-by.html)
use_tf_eager = False #@param {type:"boolean"}

# Use try/except so we can easily re-execute the whole notebook.
if use_tf_eager:
    try:
        tf.enable_eager_execution()
    except:
        pass

import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow_probability import edward2 as ed
  
def evaluate(tensors):
    """Evaluates Tensor or EagerTensor to Numpy `ndarray`s.
    Args:
    tensors: Object of `Tensor` or EagerTensor`s; can be `list`, `tuple`,
      `namedtuple` or combinations thereof.

    Returns:
      ndarrays: Object with same structure as `tensors` except with `Tensor` or
        `EagerTensor`s replaced by Numpy `ndarray`s.
    """
    if tf.executing_eagerly():
        return tf.contrib.framework.nest.pack_sequence_as(
            tensors,
            [t.numpy() if tf.contrib.framework.is_tensor(t) else t
             for t in tf.contrib.framework.nest.flatten(tensors)])
    return sess.run(tensors)

class _TFColor(object):
    """Enum of colors used in TF docs."""
    red = '#F15854'
    blue = '#5DA5DA'
    orange = '#FAA43A'
    green = '#60BD68'
    pink = '#F17CB0'
    brown = '#B2912F'
    purple = '#B276B2'
    yellow = '#DECF3F'
    gray = '#4D4D4D'
    def __getitem__(self, i):
        return [
            self.red,
            self.orange,
            self.green,
            self.blue,
            self.pink,
            self.brown,
            self.purple,
            self.yellow,
            self.gray,
        ][i % 9]
TFColor = _TFColor()

def session_options(enable_gpu_ram_resizing=True, enable_xla=True):
    """
    Allowing the notebook to make use of GPUs if they're available.
    
    XLA (Accelerated Linear Algebra) is a domain-specific compiler for linear 
    algebra that optimizes TensorFlow computations.
    """
    config = tf.ConfigProto()
    config.log_device_placement = True
    if enable_gpu_ram_resizing:
        # `allow_growth=True` makes it possible to connect multiple colabs to your
        # GPU. Otherwise the colab malloc's all GPU ram.
        config.gpu_options.allow_growth = True
    if enable_xla:
        # Enable on XLA. https://www.tensorflow.org/performance/xla/.
        config.graph_options.optimizer_options.global_jit_level = (
            tf.OptimizerOptions.ON_1)
    return config


def reset_sess(config=None):
    """
    Convenience function to create the TF graph & session or reset them.
    """
    if config is None:
        config = session_options()
    global sess
    tf.reset_default_graph()
    try:
        sess.close()
    except:
        pass
    sess = tf.InteractiveSession(config=config)

reset_sess()

# from edward.models import Gamma, Poisson, Normal, PointMass, TransformedDistribution
# from edward.util import Progbar
from observations import nips


## Introduction

This notebook demonstrates how to use a Sparse Gamma deep exponential family like the kind described in [Ranganath et al., 2015](http://proceedings.mlr.press/v38/ranganath15.pdf).



## Our example

We apply it as a topic model on the collection of NIPS 2011 conference papers.

In [0]:
#@title Hyperparameters

#@markdown Initial learning rate (default=`1e-4`)
learning_rate = 0.001 #@param  {type:"number", allow-input: true}
#@markdown Number of training steps to run (default=`200000`)
max_steps = 200000 #@param {type:"slider", min:10000, max:200000, step:1000}
#@markdown Comma-separated list denoting number of latent variables (stochastic units) per layer. (default=[`"100"`, `"30"`, `"15"`])
layer_sizes = ["100", "30", "15"] #@param {type:"raw"}
#@markdown Shape hyperparameter for Gamma priors on latents. (default=0.1)
shape = 0.1 #@param {type:"slider", min:0.1, max:1, step:0.1}
#@markdown Directory where data is stored (if using real data).
data_dir = os.path.join(os.getenv("TEST_TMPDIR", "/tmp"), "deep_exponential_family/data") #@param {type:"raw"}
#@markdown Directory to put the model's fit.
model_dir = os.path.join(os.getenv("TEST_TMPDIR", "/tmp"), "deep_exponential_family/")#@param {type:"raw"}
#@markdown If true, uses fake data. Defaults to real data (default=None)
fake_data = None #@param [None, True] {type:"raw"}

### Model

Our Model is a multi-layered topic model over a documents-by-terms matrix. We can picture it as a combination of the following nested model (based on the Gamma distribution shape parameter $k$ being provided):
$$
\textbf{Z}_2 = \text{Gamma}(\text{shape}=0.1\text{, scale}=0.1) \\
\textbf{W}_2 = \text{Gamma}(\text{shape}=0.1, \text{scale}=0.3) \\
\textbf{Z}_1 = \text{Gamma}(\text{shape}=k\text{,  scale}=k/(\textbf{Z}_2\textbf{W}_2^T)) \\
\textbf{W}_1 = \text{Gamma}(\text{shape}=0.1, \text{scale}=0.3) \\
\textbf{Z}_0 = \text{Gamma}(\text{shape}=k\text{,  scale}=k/(\textbf{Z}_1\textbf{W}_1^T)) \\
\textbf{W}_0 = \text{Gamma}(\text{shape}=0.1, \text{scale}=0.3) \\
\text{ }\\
x = \text{Poisson}(\textbf{Z}_0\textbf{W}_0^T) 
$$

In [0]:
def deep_exponential_family(data_size, feature_size, units, shape):
    """A multi-layered topic model over a documents-by-terms matrix."""
    w2 = tfd.Gamma(tf.fill([units[2], units[1]], 0.1), tf.fill([units[2], units[1]], 0.3), name="w2")
    w1 = tfd.Gamma(tf.fill([units[1], units[0]], 0.1), tf.fill([units[1], units[0]], 0.3), name="w1")
    w0 = tfd.Gamma(tf.fill([units[0], feature_size], 0.1), tf.fill([units[0], feature_size], 0.3), name="w0")

    z2 = tfd.Gamma(tf.fill([data_size, units[2]], 0.1), tf.fill([data_size, units[2]], 0.1), name="z2")
    z1 = tfd.Gamma(shape, shape / tf.matmul(z2.sample(), w2.sample()), name="z1")
    z0 = tfd.Gamma(shape, shape / tf.matmul(z1.sample(), w1.sample()), name="z0")
    x = tfd.Poisson(tf.matmul(z0.sample(), w0.sample()), name="x")
    return x


The first of these is our learnable deterministic distribution over positive reals.

In [0]:
def trainable_positive_deterministic(shape, min_loc=1e-3, name=None):
    """Learnable Deterministic distribution over positive reals."""
    with tf.variable_scope(None, default_name="trainable_positive_deterministic"):
        unconstrained_loc = tf.get_variable("unconstrained_loc", shape)
        loc = tf.maximum(tf.nn.softplus(unconstrained_loc), min_loc)
        rv = tfd.Deterministic(loc=loc, name=name)
        return rv

Next up we have the learnable gamma distribution ($\text{Gamma}(\alpha, \beta)$) via concentration and scale parameterization. We have a gamma distribution. 

In [0]:
def trainable_gamma(shape, min_concentration=1e-3, min_scale=1e-5, name=None):
    """Learnable Gamma via concentration and scale parameterization."""
    with tf.variable_scope(None, default_name="trainable_gamma"):
        unconstrained_concentration = tf.get_variable(
            "unconstrained_concentration", shape,
            initializer=tf.random_normal_initializer(mean=0.5, stddev=0.1))
        unconstrained_scale = tf.get_variable(
            "unconstrained_scale", shape,
            initializer=tf.random_normal_initializer(stddev=0.1))
        concentration = tf.maximum(tf.nn.softplus(unconstrained_concentration),
                                   min_concentration)
        rate = tf.maximum(1. / tf.nn.softplus(unconstrained_scale), 1. / min_scale)
        rv = tfd.Gamma(concentration=concentration, rate=rate, name=name)
        return rv

We can define the Posterior approximation for deep exponential family, which we can define as follows:
$$p(w_{0,1,2}, z_{1,2,3} | x)$$

In [0]:
def deep_exponential_family_variational(data_size, feature_size, units):
    """Posterior approx. for deep exponential family p(w{0,1,2}, z{1,2,3} | x)."""
    qw2 = trainable_positive_deterministic([units[2], units[1]], name="qw2")
    qw1 = trainable_positive_deterministic([units[1], units[0]], name="qw1")
    qw0 = trainable_positive_deterministic([units[0], feature_size], name="qw0")
    qz2 = trainable_gamma([data_size, units[2]], name="qz2")
    qz1 = trainable_gamma([data_size, units[1]], name="qz1")
    qz0 = trainable_gamma([data_size, units[0]], name="qz0")
    return qw2, qw1, qw0, qz2, qz1, qz0

### Data

The NIPS 1987-2015 data set is in the form of a $ 11,463 \times 5,812$ matrix of per-paper word counts, containing $11,463$ words and $5,811$ NIPS conference papers (Perrone et al., 2016). We subset to papers in 2011 and words appearing in at least two documents and having a total word count of at least 10. Built from the Observations Python package.

In [0]:
def load_nips2011_papers(path):
    """Loads NIPS 2011 conference papers.
    The NIPS 1987-2015 data set is in the form of a 11,463 x 5,812 matrix of
    per-paper word counts, containing 11,463 words and 5,811 NIPS conference
    papers (Perrone et al., 2016). We subset to papers in 2011 and words appearing
    in at least two documents and having a total word count of at least 10.
    Built from the Observations Python package.
    Args:
        path: str.
          Path to directory which either stores file or otherwise file will
          be downloaded and extracted there. Filename is `NIPS_1987-2015.csv`.
    Returns:
        bag_of_words: np.ndarray of shape [num_documents, num_words]. Each element
          denotes the number of occurrences of a specific word in a specific
          document.
        words: List of strings, denoting the words for `bag_of_words`'s columns.
    """
    path = os.path.expanduser(path)
    filename = "NIPS_1987-2015.csv"
    filepath = os.path.join(path, filename)
    if not os.path.exists(filepath):
        url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/"
                 "00371/NIPS_1987-2015.csv")
        if not tf.gfile.Exists(path):
            tf.gfile.MakeDirs(path)
        print("Downloading %s to %s" % (url, filepath))
        urllib.request.urlretrieve(url, filepath)

    with open(filepath) as f:
        iterator = csv.reader(f)
        documents = next(iterator)[1:]
        words = []
        x_train = []
        for row in iterator:
            words.append(row[0])
            x_train.append(row[1:])

    x_train = np.array(x_train, dtype=np.int)

    # Subset to documents in 2011 and words appearing in at least two documents
    # and have a total word count of at least 10.
    doc_idx = [i for i, document in enumerate(documents)
               if document.startswith("2011")]
    documents = [documents[doc] for doc in doc_idx]
    x_train = x_train[:, doc_idx]
    word_idx = np.logical_and(np.sum(x_train != 0, 1) >= 2,
                              np.sum(x_train, 1) >= 10)
    words = [word for word, idx in zip(words, word_idx) if idx]
    bag_of_words = x_train[word_idx, :].T
    return bag_of_words, words

In [0]:
layer_sizes = [int(layer_size) for layer_size in layer_sizes]
if len(layer_sizes) != 3:
    raise NotImplementedError("Specifying fewer or more than 3 layers is not "
                              "currently available.")
if tf.gfile.Exists(model_dir):
    tf.logging.warning(
        "Warning: deleting old log directory at {}".format(model_dir))
    tf.gfile.DeleteRecursively(model_dir)
tf.gfile.MakeDirs(model_dir)

if fake_data:
    bag_of_words = np.random.poisson(1., size=[10, 25])
    words = [str(i) for i in range(25)]
else:
    bag_of_words, words = load_nips2011_papers(data_dir)

total_count = np.sum(bag_of_words)
bag_of_words = tf.to_float(bag_of_words)
data_size, feature_size = bag_of_words.shape

In [0]:
print("total_count: {}".format(total_count))
print("bag_of_words: {}".format(bag_of_words))
print("data_size: {}".format(data_size))
print("feature_size: {}".format(feature_size))

total_count: 669197
bag_of_words: Tensor("ToFloat:0", shape=(305, 5970), dtype=float32)
data_size: 305
feature_size: 5970


### Inference

In [0]:
# Compute expected log-likelihood. First, sample from the variational
# distribution; second, compute the log-likelihood given the sample.
qw2, qw1, qw0, qz2, qz1, qz0 = deep_exponential_family_variational(
    data_size,
    feature_size,
    layer_sizes)

posterior_predictive = deep_exponential_family(data_size=data_size, 
                                               feature_size=feature_size,
                                               units=layer_sizes, 
                                               shape=shape)

log_likelihood = posterior_predictive.log_prob(bag_of_words)
log_likelihood = tf.reduce_sum(log_likelihood)
tf.summary.scalar("log_likelihood", log_likelihood);

In [0]:
# Defining our prior distributions

w2_prior = tfd.Gamma(tf.fill([layer_sizes[2], layer_sizes[1]], 0.1), 
                     tf.fill([layer_sizes[2], layer_sizes[1]], 0.3), 
                     name="w2_prior")
w1_prior = tfd.Gamma(tf.fill([layer_sizes[1], layer_sizes[0]], 0.1), 
                     tf.fill([layer_sizes[1], layer_sizes[0]], 0.3), 
                     name="w1_prior")
w0_prior = tfd.Gamma(tf.fill([layer_sizes[0], feature_size], 0.1), 
                     tf.fill([layer_sizes[0], feature_size], 0.3), 
                     name="w0_prior")
z2_prior = tfd.Gamma(tf.fill([data_size, layer_sizes[2]], 0.1), 
                     tf.fill([data_size, layer_sizes[2]], 0.1), 
                     name="z2_prior")
z1_prior = tfd.Gamma(shape, 
                     shape / tf.matmul(z2_prior.sample(), w2_prior.sample()), 
                     name="z1_prior")
z0_prior = tfd.Gamma(shape, 
                     shape / tf.matmul(z1_prior.sample(), w1_prior.sample()), 
                     name="z0_prior")

# Compute analytic KL-divergence between variational and prior distributions.
kl = 0.
kl += tf.reduce_sum(qz0.kl_divergence(z0_prior))
kl += tf.reduce_sum(qz1.kl_divergence(z1_prior))
kl += tf.reduce_sum(qz2.kl_divergence(z2_prior))
kl += tf.reduce_sum(qw0.kl_divergence(w0_prior))
kl += tf.reduce_sum(qw1.kl_divergence(w1_prior))
kl += tf.reduce_sum(qw2.kl_divergence(w2_prior))

tf.summary.scalar("kl", kl);

The Log-likelihood and Kullback–Leibler divergence are then combined into the Evidence Lower BOund (ELBO) loss. We will minimize the ELBO using the Adam Algorithm ([Kingma et al., 2014](https://arxiv.org/abs/1412.6980)).

In [0]:
elbo = log_likelihood - kl
tf.summary.scalar("elbo", elbo)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(-elbo)


Now we can finally run our inference. 

**NOTE**: The loss function can sometimes erroneously output a negative value or `NaN`. This happens when the samples from the variational approximation are numerically zero, which causes Gamma log probs to output `inf`.

In [0]:
summary = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(model_dir, sess.graph)
start_time = time.time()

evaluate(tf.global_variables_initializer())
for step in range(max_steps):
    start_time = time.time()
    _, elbo_value = evaluate([train_op, elbo])
    if step % 500 == 0:
        duration = time.time() - start_time
        print("Step: {:>3d} Loss: {:.3f} ({:.3f} sec)".format(
              step, elbo_value, duration))
        summary_str = evaluate(summary)
        summary_writer.add_summary(summary_str, step)
        summary_writer.flush()

        # Compute perplexity of the full data set. The model's negative
        # log-likelihood of data is upper bounded by the variational objective.
        negative_log_likelihood = -elbo_value
        perplexity = np.exp(negative_log_likelihood / total_count)
        print("Negative log-likelihood <= {:0.3f}".format(
              negative_log_likelihood))
        print("Perplexity <= {:0.3f}".format(perplexity))

        # Print top 10 words for first 10 topics.
        qw0_values = evaluate(qw0.sample())
        for k in range(min(10, layer_sizes[-1])):
            top_words_idx = qw0_values[k, :].argsort()[-10:][::-1]
            top_words = " ".join([words[i] for i in top_words_idx])
            print("Topic {}: {}".format(k, top_words))

Step:   0 Loss: -2501358080.000 (0.726 sec)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp


Negative log-likelihood <= 2501358080.000
Perplexity <= inf
Topic 0: reward alarm conference advance highlighted projecting similarly sources semantics date
Topic 1: write dan optimised advantages category opportunity issues play aggregated tries
Topic 2: comprehensive forms cessing valuable ancestors tails critic indexing raises phys
Topic 3: shortly black accomplished instead specifies rise invariance lausanne shrink maintain
Topic 4: orders saw regressors phrase star observes sapiro impression morgan formal
Topic 5: agglomerative together separation tenenbaum cauchy interscience animals environment mild berkeley
Topic 6: syst describing tendency completed opposed capable lambda restaurant riding prevalence
Topic 7: location segments lemmas static treated connected generative sphere diagnostic contribution
Topic 8: speaker darpa refinement schneider shamir adversarial vary altun savings may
Topic 9: platform clearly overhead ibp ronald evaluations extensions jacobian typically chapel

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp


Topic 0: reward alarm conference advance highlighted projecting similarly sources semantics date
Topic 1: write dan optimised advantages category opportunity issues play aggregated tries
Topic 2: comprehensive forms cessing valuable ancestors tails critic indexing raises phys
Topic 3: shortly black accomplished instead specifies rise invariance lausanne shrink maintain
Topic 4: orders saw regressors phrase star observes sapiro impression morgan formal
Topic 5: agglomerative together separation tenenbaum cauchy interscience animals environment mild berkeley
Topic 6: syst describing tendency completed opposed capable lambda restaurant riding prevalence
Topic 7: location segments lemmas static treated connected generative sphere diagnostic contribution
Topic 8: speaker darpa refinement schneider shamir adversarial vary altun savings may
Topic 9: platform clearly overhead ibp ronald evaluations extensions jacobian typically chapelle
Step: 1000 Loss: -3812073984.000 (0.196 sec)
Negative log

...

Topic 0: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 1: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 2: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 3: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 4: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 5: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 6: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 7: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 8: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
T

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp


Topic 0: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 1: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 2: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 3: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 4: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 5: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 6: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 7: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 8: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
T

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp


Topic 0: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 1: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 2: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 3: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 4: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 5: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 6: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 7: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
Topic 8: zurich fails factorisation factorization factorizations factorize factorized factorizes factors facts
T

## Conclusions

As we can see, with the decreasing negative log-likelihood we get lists of words that seem to be more intuitively separable in to the various topics. Granted, these topics don't have names themselves, but this separation is clear.

With default settings (in particular, with $\text{log-Normal}$ variational approximation), this model takes ~62 seconds per epoch on a Titan X (Pascal). The following results are on epoch 12.

**Topic 0:** `let distribution set strategy distributions given learning information use property`
    
**Topic 1:** `functions problem risk function submodular cut level clustering sets performance`

**Topic 2:** `action value learning regret reward actions algorithm optimal state return`

**Topic 3:** `posterior stochastic approach information based using prior
    mean divergence since`

**Topic 4:** `player inference game propagation experts static query expert
    base variables`

**Topic 5:** `algorithm set loss weak algorithms optimal submodular online
    cost setting`

**Topic 6:** `sparse sparsity norm solution learning penalty greedy
    structure wise regularization`

**Topic 7:** `learning training linear kernel using coding accuracy
    performance dataset based`

**Topic 8:** `object categories image features examples classes images
    class objects visual`

**Topic 9:** `data manifold matrix points dimensional point low linear
    gradient optimization`

If we use a $\text{Gamma}$ variational approximation (instead of a $\text{log-Normal}$ approximation) , this produces worse results. This is likely due to the high variance in stochastic gradients. The $\text{Gamma}$ version takes ~2 minutes per epoch on a Titan X (Pascal). This is what the results look like on epoch 12.

**Topic 0:** `reasons posterior tion using similar tools university input computed refers`

**Topic 1:** `expected since much related rate defined optimization vector thus neurons`

**Topic 2:** `large linear given table shown true drop classification constraints current`

**Topic 3:** `proposed processing estimated better values gaussian form test true setting`

**Topic 4:** `see methods local several rate processing general vector enables section`

**Topic 5:** `thus case methods image dataset models different instead new respectively`

**Topic 6:** `based consider samples step object see kernel since problem training`

**Topic 7:** `approaches linear computing show gaussian data expected analysis well proof`

**Topic 8:** `fig point kernel bayesian solution applications results follows regression computer`

**Topic 9:** `conference optimization training pages maximum learning dataset performance state inference`

## Reference

[1] Michael Figurnov, Shakir Mohamed, Andriy Mnih. ["Implicit Reparameterization Gradients, 2018"](https://arxiv.org/abs/1805.08498). https://arxiv.org/abs/1805.08498.
     
[2] Valerio Perrone and Paul A Jenkins and Dario Spano and Yee Whye Teh. [Poisson Random Fields for Dynamic Feature Models](https://arxiv.org/abs/1611.07460), 2016. https://arxiv.org/abs/1611.07460
     
[3] Rajesh Ranganath, Linpeng Tang, Laurent Charlin, David M. Blei. [Deep exponential families.](http://www.cs.toronto.edu/~lcharlin/papers/def_aistats.pdf) In _Artificial Intelligence and Statistics_, 2015. https://arxiv.org/abs/1411.2581

Other code examples:
- [The existing (Edward2) TFP Example](https://github.com/tensorflow/probability/blob/22cb49d4b7d70a96a46f3a230792eab7ef793cb8/tensorflow_probability/examples/deep_exponential_family.py)
- [Deep Exponential Family in Gluon](https://github.com/altosaar/deep-exponential-families-gluon)
- [A fork of the Blei Lab example](https://github.com/mkturkcan/DEF-val/blob/dbaf43c8b76f789e9630b9d8b6085cae463cbc9b/DEF-val.py)

